In [1]:
import os 
import pandas as pd
import numpy as np
import math

import geopandas as gpd
import json

from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler

output_notebook()

Loading BokehJS ...

In [2]:
#path = r'C:/Users/ShrekTheOger/Documents/GitHub/final-project-final-project-bowen-and-natasia'
path = r'C:\Users\engel\Documents\GitHub\final-project-final-project-bowen-and-natasia'
#import warnings
#warnings.filterwarnings('ignore')

In [3]:
#Load informational dataframe from refined file folder
df = os.path.join(path+'/refined_data', 'df_income_pop.csv')
df = pd.read_csv(df)

In [4]:
#retrive shape file of us in county level
county_shp = os.path.join(path+'/raw_data', 'cb_2020_us_county_20m', 'cb_2020_us_county_20m.shp')
county = gpd.read_file(county_shp)

In [5]:
#Combining state_id with county_id into same format as in university shape file
l = []
for i in range(len(df['state_id'])):
    if len(str(df['state_id'][i])) == 1:
        if len(str(df['county_id'][i])) == 1:
            a = '0'+ str(df['state_id'][i]) + '00' + str(df['county_id'][i])
        elif len(str(df['county_id'][i])) == 2:
            a = '0'+ str(df['state_id'][i]) + '0' + str(df['county_id'][i])
        else:
            a = '0'+ str(df['state_id'][i]) + str(df['county_id'][i])
    else:
        if len(str(df['county_id'][i])) == 1:
            a = str(df['state_id'][i]) + '00' + str(df['county_id'][i])
        elif len(str(df['county_id'][i])) == 2:
            a = str(df['state_id'][i]) + '0' + str(df['county_id'][i])
        else:
            a = str(df['state_id'][i]) + str(df['county_id'][i])
    l.append(a)
df.insert(3, 'GEOID', l)
df.drop(labels=['state_id', 'county_id'], axis=1, inplace = True)


In [6]:
county.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,STATE_NAME,LSAD,ALAND,AWATER,geometry
0,01,061,00161556,0500000US01061,01061,Geneva,Geneva County,AL,Alabama,06,1487908432,11567409,"POLYGON ((-86.19348 31.19221, -86.12541 31.182..."
1,08,125,00198178,0500000US08125,08125,Yuma,Yuma County,CO,Colorado,06,6123763559,11134665,"POLYGON ((-102.80377 40.00255, -102.79358 40.3..."
2,17,177,01785076,0500000US17177,17177,Stephenson,Stephenson County,IL,Illinois,06,1461392061,1350223,"POLYGON ((-89.92647 42.50579, -89.83759 42.504..."
3,28,153,00695797,0500000US28153,28153,Wayne,Wayne County,MS,Mississippi,06,2099745602,7255476,"POLYGON ((-88.94335 31.82456, -88.91046 31.826..."
4,34,041,00882237,0500000US34041,34041,Warren,Warren County,NJ,New Jersey,06,923435921,15822933,"POLYGON ((-75.19261 40.71587, -75.17748 40.764..."


In [7]:
df.head()

,year,state,county,GEOID,COUNTYFIPS,total_population,total_native,total_born_in_state,total_born_out_state,total_born_outside_US,total_foreign_born,income_past12m
0,2010,California,Merced County,06047,6047,250699,188498,156434,29768,2296,62201,18041.0
1,2010,California,Modoc County,06049,6049,9605,8971,6067,2882,22,634,20536.0
2,2010,California,Mono County,06051,6051,13905,11341,7059,4108,174,2564,27321.0
3,2010,California,Monterey County,06053,6053,407435,284647,208218,69900,6529,122788,25776.0
4,2010,California,Napa County,06055,6055,134051,103891,73787,27759,2345,30160,34310.0


In [8]:
df['GEOID2'] = df['COUNTYFIPS'].apply(lambda x: '{0:05}'.format(x))

In [9]:
df.dtypes

year                       int64
state                     object
county                    object
GEOID                     object
COUNTYFIPS                 int64
total_population           int64
total_native               int64
total_born_in_state        int64
total_born_out_state       int64
total_born_outside_US      int64
total_foreign_born         int64
income_past12m           float64
GEOID2                    object
dtype: object

In [10]:
df.tail()

,year,state,county,GEOID,COUNTYFIPS,total_population,total_native,total_born_in_state,total_born_out_state,total_born_outside_US,total_foreign_born,income_past12m,GEOID2
31069,2019,Tennessee,Crockett County,47033,47033,14399,13774,11231,2490,53,625,23771.0,47033
31070,2019,Tennessee,Lake County,47095,47095,7401,7307,5436,1839,32,94,15732.0,47095
31071,2019,Tennessee,Knox County,47093,47093,461104,438362,272843,161946,3573,22742,33229.0,47093
31072,2019,Washington,Benton County,53005,53005,197518,176385,104182,70549,1654,21133,32882.0,53005
31073,2019,Washington,Clark County,53011,53011,473252,423305,165377,251903,6025,49947,35860.0,53011


In [11]:
#Combine yearly data of county level income and population data with county geometries
df_shape = county.merge(df, how = 'inner', on='GEOID').set_geometry('geometry')

In [12]:
#retrive shape file of universities
uni_df = os.path.join(path+'/refined_data', 'uni_fund_df.csv')
uni_df = pd.read_csv(uni_df)

In [13]:
universities_shp = os.path.join(path+'/raw_data', 'Colleges_and_Universities-shp', 'Colleges_and_Universities.shp')
universities = gpd.read_file(universities_shp)

In [14]:
universities['IPEDSID'] = universities['IPEDSID'].astype(str).astype(int)
uni_shape = universities.merge(uni_df, how = 'inner', on = 'IPEDSID').set_geometry('geometry')
uni_shape = uni_shape.to_crs(df_shape.crs)

In [15]:
df_shape['year']

0        2010
1        2011
2        2012
3        2013
4        2014
         ... 
31060    2015
31061    2016
31062    2017
31063    2018
31064    2019
Name: year, Length: 31065, dtype: int64

# Interactive Geographic Map Using Bokeh
Here we use Bokeh to display county level population information with hovering and display income discrepensies using density map.
We also displayed where the top 50 universities at each state, also displaying their fund level using hovering effect.

Our code is heavily inspired by Jim King from his website https://jimking100.github.io/2019-09-04-Post-3/

In [16]:
def json_data(selectedYear, selectedState):
    yr = selectedYear
    st = selectedState
    # Pull selected year from state data
    df_yr = df_shape[(df_shape['year'] == yr) & (df_shape['state'] == st)].copy().reset_index(drop=True)
    
    merged_json = json.loads(df_yr.to_json())
    
    # Convert to json preferred string-like object 
    json_data = json.dumps(merged_json)
    return json_data

In [17]:
def json_data_uni(selectedYear, selectedState):
    yr = selectedYear
    st = selectedState
    # Pull selected year from state data
    uni_yr = uni_shape[(uni_shape['year'] == yr) & (uni_shape['state'] == st)].copy().reset_index(drop=True)
    merged_json = json.loads(uni_yr.to_json())
    
    # Convert to json preferred string-like object 
    json_data = json.dumps(merged_json)
    return json_data

In [18]:
"""The disadvantage of using a static scale is some county like Los Angelous has a overly dense population which pulled
the scale upper limit really high, so when we have state level inspections, many counties that have less than 100,000 become 
insignificant, even those big counties like Cook County become drawfted by the abnormal dense population from city like New 
York and Los Angeles."""


# This dictionary contains the formatting for the data in the plots
format_data = [('total_population', 0, 12000000,'0,0', 'Total Population'),
               ('total_native', 0, 6750000,'0,0', 'Total Native'),
               ('total_born_in_state', 0, 5200000,'0,0', 'Total Born in State'),
               ('total_born_out_state', 0, 2000000,'0,0', 'Total Born out State'),
               ('total_born_outside_us', 0, 130000,'0,0', 'Total Born out State'),
               ('total_foreign_born', 0, 3500000,'0,0', 'Total Foreigner'),
               ('income_past12m', 0, 80000,'$0,0', 'Income in Past 12 Months') 
              ]
 
#Create a DataFrame object from the dictionary 
format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

In [19]:
# Define the callback function: update_plot
def update_plot(attr, old, new):
    # The input yr is the year selected from the slider
    yr = slider.value
    print(yr)
    st = select_st.value
    print(st)
    new_data = json_data(yr, st)
    new_uni = json_data_uni(yr, st)

    # The input cr is the criteria selected from the select box
    input_field = select.value
    input_field = format_df.loc[format_df['verbage'] == input_field, 'field'].iloc[0]
    
    # Update the plot based on the changed inputs
    p = make_plot(input_field)
    
    # Update the layout, clear the old document and display the new document
    layout = column(p, select, select_st, slider)
    curdoc().clear()
    curdoc().add_root(layout)

    # Update the data
    geosource.geojson = new_data
    geosource_uni.geojson = new_uni


In [20]:
# Define the callback function: update_plot
def update_plot_uni(attr, old, new):
    # The input yr is the year selected from the slider
    yr = slider.value
    st = select_st.value
    new_uni = json_data_uni(yr, st)

    # Update the data
    geosource_uni.geojson = new_uni

In [21]:
#def update_plot_all(attr, old, new):
#    
#    update_plot(attr, old, new)
#    update_plot_uni(attr, old, new)
    

In [22]:
# Create a plotting function
def make_plot(field_name):  
    # Set the format of the colorbar
    min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
    max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
    field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]

    # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)
    
    # Create color bar.
    format_tick = NumeralTickFormatter(format=field_format)
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, border_line_color=None, location = (0, 0))

    # Create figure object.
    verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]

    p = figure(title = verbage, 
               plot_height = 650, plot_width = 850)
    
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False

    # Add patch renderer to figure. 
    r1 = p.patches('xs','ys', source = geosource, fill_color = {'field' : field_name, 'transform' : color_mapper},
                   line_color = 'black', line_width = 0.25, fill_alpha = 1)

    # Specify color bar layout.
    p.add_layout(color_bar, 'right')

    # Add the hover tool to the graph
    p.add_tools(HoverTool(renderers=[r1], tooltips=hover))
    
    r2 = p.circle('x','y', color = 'red', source=geosource_uni, size=5, fill_alpha = 0.7)
    p.add_tools(HoverTool(renderers=[r2], tooltips=hover_uni))
    
    return p

In [26]:
# Input geojson source that contains features for plotting for:
# initial year 2010 and initial income per capital for counties -- Income_past12m
geosource = GeoJSONDataSource(geojson = json_data(2015,'Illinois'))
geosource_uni = GeoJSONDataSource(geojson = json_data_uni(2015,'Illinois'))
input_field = 'income_past12m'

# Define a sequential multi-hue color palette.
palette = brewer['Blues'][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

# Add hover tool

hover = [ ('County','@NAME'),('State', '@state'),('Total population', '@total_population{,}'),
         ('Born instate', '@total_born_in_state{,}'),('Born outstate', '@total_born_out_state{,}'),]
hover_uni = [ ('School','@NAME'),('Fund', '@fund{,}')]


# Call the plotting function
p = make_plot(input_field)

# Make a slider object: slider 
slider = Slider(title = 'Year',start = 2010, end = 2019, step = 1, value = 2015)


# Make a selection object: select
select = Select(title='Select Criteria:', value='income_past12m', options=['Income in Past 12 Months', 'Total Population'])

# Make a selection object: select
select_st = Select(title='Select Target State:', value='Illinois', options=uni_shape['state'].unique().tolist())

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
# Display the current document
layout = column(p, select, select_st, slider)


def modify_doc(doc):
    doc.add_root(column(layout))
    slider.on_change('value', update_plot)
    select.on_change('value', update_plot)
    select_st.on_change('value', update_plot)
    slider.on_change('value', update_plot_uni)
    select.on_change('value', update_plot_uni)
    select_st.on_change('value', update_plot_uni)
    

handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app)

2015
Illinois
2015
Virginia
2016
Virginia
2016
Virginia
2016
California
